In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist # Keras  Fashion-MNIST
import matplotlib.pyplot as plt
import time  # added for timing
import pandas as pd
from google.colab import files

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# Define different parameter sets
#configs = [...] # (as before) Random choices

#after intial testing, the cofigurations have been re-ordered by rank of Test Accuracy
configs = [
    # Adam (3 versions)
    {'filters': 64, 'kernel_size': 5, 'dense_units': 128, 'optimizer': 'adam',    'batch_size': 64, 'learning_rate': 0.001},
    {'filters': 32, 'kernel_size': 3, 'dense_units': 128, 'optimizer': 'adam',    'batch_size': 128,'learning_rate': 0.0005},
    {'filters': 64, 'kernel_size': 3, 'dense_units': 64,  'optimizer': 'adam',    'batch_size': 64, 'learning_rate': 0.0001},

    # RMSprop (3 versions)
    {'filters': 64, 'kernel_size': 5, 'dense_units': 128, 'optimizer': 'rmsprop', 'batch_size': 64, 'learning_rate': 0.001},
    {'filters': 32, 'kernel_size': 3, 'dense_units': 64,  'optimizer': 'rmsprop', 'batch_size': 64, 'learning_rate': 0.0005},
    {'filters': 32, 'kernel_size': 5, 'dense_units': 128, 'optimizer': 'rmsprop', 'batch_size': 128,'learning_rate': 0.0001},

    # SGD (3 versions)
    {'filters': 64, 'kernel_size': 3, 'dense_units': 128, 'optimizer': 'sgd',     'batch_size': 64, 'learning_rate': 0.01},
    {'filters': 32, 'kernel_size': 5, 'dense_units': 128, 'optimizer': 'sgd',     'batch_size': 128,'learning_rate': 0.01},
    {'filters': 64, 'kernel_size': 3, 'dense_units': 64,  'optimizer': 'sgd',     'batch_size': 64, 'learning_rate': 0.005},
]


results = []
all_train_acc = []
all_val_acc = []

# Loop through each config
for i, cfg in enumerate(configs, 1):
    print(f"\nTraining configuration {i}: {cfg}")

    model = models.Sequential([
        layers.Input(shape=x_train.shape[1:]),  # Define the input explicitly
        layers.Conv2D(cfg['filters'], (cfg['kernel_size'], cfg['kernel_size']), activation='relu'),
        layers.MaxPooling2D((2, 2)),            # pool on a 2x2 basis
        layers.Flatten(),                       # flaten to a vector
        layers.Dense(cfg['dense_units'], activation='relu'), # array of neurons
        layers.Dense(10, activation='softmax')  # one unit per class
    ])

    # Define optimizer with custom learning rate
    if cfg['optimizer'] == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=cfg['learning_rate'])
    elif cfg['optimizer'] == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=cfg['learning_rate'])
    elif cfg['optimizer'] == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=cfg['learning_rate'])
    else:
        raise ValueError(f"Unsupported optimizer: {cfg['optimizer']}")
    
    model.compile(optimizer=cfg['optimizer'],
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    start_time = time.time()  # Start timing
    history = model.fit(x_train, y_train,
                        epochs=5,
                        batch_size=cfg['batch_size'],
                        validation_split=0.1,
                        verbose=0)
    end_time = time.time()  # End timing
    duration = round(end_time - start_time, 2)

    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print(f"Test accuracy: {test_acc:.4f}")

    results.append({
        'config': cfg,
        'test_accuracy': test_acc,
        'train_accuracy': history.history['accuracy'],
        'val_accuracy': history.history['val_accuracy'],
        'training_time_sec': duration  #  added
    })

    # Plot training history for each
    plt.figure(figsize=(5, 3))
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title(f"Accuracy (Config {i})")
    plt.xlabel("Epoch")
    plt.xticks(range(len(history.history['accuracy'])))
    plt.ylabel("Accuracy")
    plt.ylim(0.68, 0.95)
    plt.legend()
    plt.tight_layout()
    plt.show()

# All training accuracy curves
plt.figure(figsize=(6, 4))
for i, r in enumerate(results):
    plt.plot(r['train_accuracy'], label=f"Train {i+1}")
plt.title("All Training Accuracy Curves")
plt.xlabel("Epoch")
plt.xticks(range(len(results[0]['train_accuracy'])))
plt.ylabel("Accuracy")
plt.ylim(0.68, 0.95)
plt.legend()
plt.tight_layout()
plt.show()

# All validation accuracy curves
plt.figure(figsize=(6, 4))
for i, r in enumerate(results):
    plt.plot(r['val_accuracy'], label=f"Val {i+1}")
plt.title("All Validation Accuracy Curves")
plt.xlabel("Epoch")
plt.xticks(range(len(results[0]['val_accuracy'])))
plt.ylabel("Accuracy")
plt.ylim(0.68, 0.95)
plt.legend()
plt.tight_layout()
plt.show()

# Create a summary DataFrame
summary = []
for i, r in enumerate(results, 1):
    summary.append({
        'Config #': i,
        'Filters': r['config']['filters'],
        'Kernel': r['config']['kernel_size'],
        'Dense Units': r['config']['dense_units'],
        'Optimizer': r['config']['optimizer'],
        'Batch Size': r['config']['batch_size'],
        'Learning Rate': r['config']['learning_rate'],
        'Train Time (s)': r['training_time_sec'],
        'Train Acc (final)': round(r['train_accuracy'][-1], 4),
        'Val Acc (final)': round(r['val_accuracy'][-1], 4),
        'Test Acc': round(r['test_accuracy'], 4)
    })

summary_df = pd.DataFrame(summary)
print("\nModel Configuration Summary Table:")
display(summary_df)

#  Save to CSV and offer download
summary_df.to_csv("model_summary.csv", index=False)
files.download("model_summary.csv")
